# Notebook de conception de Notebook

Ce Notebook .Net interactive a pour objectif de permettre la création assistée d'autres notebooks .Net interactive en confiant le soin à ChatGPT d'analyser et de proposer des modifications d'une version courante, et en prenant en charge la mise à jour et l'exécution des mises à jour en function calling Open AI grâce à l'API .Net interactive. 


### 1. Initialisation

On installe des packages pour la manipulation de notebook et pour l'orchestration de LLMs.

In [1]:
// #r "nuget: Microsoft.DotNet.Interactive, *-*"
#r "nuget: Microsoft.DotNet.Interactive.CSharp, *-*"
#r "nuget: Microsoft.DotNet.Interactive.Documents, *-*"
#r "nuget: Microsoft.DotNet.Interactive.PackageManagement, *-*"
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, *-*"

### Importation des fichiers `.cs`

Un certain nombre de classes ont été conçues pour fournir l'exécution et la mise à jour de notebook .Net interactive à l'aide de Semantic-Kernel.

In [11]:
// Load some helper functions to load values from settings.json
#!import ../Config/Settings.cs 

#!import DisplayLogger.cs
#!import DisplayLoggerProvider.cs
#!import NotebookExecutor.cs
// #!import WorkbookInteractionBase.cs
// #!import WorkbookUpdateInteraction.cs
// #!import WorkbookValidation.cs
// #!import NotebookUpdaterBase.cs
// #!import AutoInvokeSKAgentsNotebookUpdater.cs

// #!import NotebookPlannerUpdater.cs
// #!import AutoGenNotebookUpdater.cs

- **Imports des espaces de noms**

On prend soin de distinguer le kernel d'exécution de notebook .Net interactive, et le kernel de semantic-kernel.

In [3]:
using Microsoft.DotNet.Interactive;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Connectors.OpenAI;
  
using System;
using System.IO;
using System.Threading.Tasks;

using Microsoft.Extensions.Logging;
using Microsoft.Extensions.DependencyInjection;

using SKernel = Microsoft.SemanticKernel.Kernel;
using IKernel = Microsoft.DotNet.Interactive.Kernel;

### 2. Mode de Fourniture des Informations

On permet à l'utilisateur de saisir les informations décrivant la tâche à accomplir dans le notebook de travail de plusieurs façons différentes.

In [4]:
public enum InformationMode
{
    Variable,
    Prompt,
    File
}



### 5. Recueil des informations

Selon le mode de fourniture des informations choisi, on récupère la tâche à accomplir dans le notebook de travail.

In [5]:
var infoCollectionDisplay = display("Collecte d'informations en cours...");

var mode = InformationMode.Variable;

string taskDescription = "Créer un notebook .Net interactive permettant de requêter DBPedia, utilisant les package Nuget dotNetRDF et XPlot.Plotly.Interactive. Commencer par éditer les cellules de Markdown pour définir et affiner l'exemple de requête dont le graphique final sera le plus pertinent. Choisir un exemple de requête complexe, manipulant des aggrégats, qui pourront être synthétisés dans un graphique. Une fois les cellules de code alimentées et les bugs corrigés, s'assurer que la sortie de la cellule générant le graphique est conforme et pertinente.";

if (mode == InformationMode.Variable)
{
    display("Utilisation de la variable pour la description de la tâche.");
}
else if (mode == InformationMode.Prompt)
{
    var questions = new[]
    {
        "Bonjour! Veuillez fournir une brève description de la tâche à accomplir.",
        "Quels sont les principaux objectifs de cette tâche?",
        "Y a-t-il des contraintes ou des conditions spécifiques à prendre en compte?",
        "Des informations supplémentaires que vous souhaitez ajouter?"
    };

    taskDescription = string.Empty;
    foreach (var question in questions)
    {
        var response = await IKernel.GetInputAsync(question);
        taskDescription += $"{question}\\n{response}\\n\\n";
    }
}


display("Informations recueillies :\\n" + taskDescription);

### 3. Personnalisation du Notebook de Travail

On charge un notebook template contenant des parties de Markdown et de code à compléter, et on injecte la tâche dans la partie descriptive en entête du notebook .Net interactive.

In [6]:

var notebookPath = @$"./Workbooks/Workbook-{DateTime.Now.ToFileTime()}.ipynb";


### Exécution et Mise à Jour Itérative avec AutoInvokeSKAgentsNotebookUpdater


In [7]:
var logger = new DisplayLogger("NotebookUpdater", LogLevel.Trace);
var updater = new AutoInvokeSKAgentsNotebookUpdater(notebookPath, logger);
updater.SetStartingNotebookFromTemplate(taskDescription);
display("Appel à UpdateNotebookWithAutoInvokeSKAgents...");
await updater.UpdateNotebookWithAutoInvokeSKAgents();
display("Mise à jour du notebook terminée.");
